In [1]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize)

In [2]:
from ISLP import confusion_table
from ISLP.models import contrast
from sklearn. discriminant_analysis import \
    (LinearDiscriminantAnalysis as LDA,
     QuadraticDiscriminantAnalysis as QDA)
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

In [3]:
Smarket = load_data("Smarket")
Smarket

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
0,2001,0.381,-0.192,-2.624,-1.055,5.010,1.19130,0.959,Up
1,2001,0.959,0.381,-0.192,-2.624,-1.055,1.29650,1.032,Up
2,2001,1.032,0.959,0.381,-0.192,-2.624,1.41120,-0.623,Down
3,2001,-0.623,1.032,0.959,0.381,-0.192,1.27600,0.614,Up
4,2001,0.614,-0.623,1.032,0.959,0.381,1.20570,0.213,Up
...,...,...,...,...,...,...,...,...,...
1245,2005,0.422,0.252,-0.024,-0.584,-0.285,1.88850,0.043,Up
1246,2005,0.043,0.422,0.252,-0.024,-0.584,1.28581,-0.955,Down
1247,2005,-0.955,0.043,0.422,0.252,-0.024,1.54047,0.130,Up
1248,2005,0.130,-0.955,0.043,0.422,0.252,1.42236,-0.298,Down


In [4]:
allvars = Smarket.columns.drop(['Today', 'Direction', 'Year'])
design = MS(allvars)
X = design.fit_transform(Smarket)
y = Smarket.Direction == 'Up'

In [5]:
train = (Smarket.Year < 2005)
Smarket_train = Smarket.loc[train]
Smarket_test = Smarket.loc[~train]
Smarket_test.shape

(252, 9)

In [6]:
X_train , X_test = X.loc[train], X.loc[~train]
y_train , y_test = y.loc[train], y.loc[~train]

In [7]:
D = Smarket.Direction
L_train , L_test = D.loc[train], D.loc[~train]

In [8]:
NB = GaussianNB ()
NB.fit(X_train , L_train)

GaussianNB()

In [9]:
NB.classes_

array(['Down', 'Up'], dtype='<U4')

In [10]:
NB.class_prior_

array([0.49198397, 0.50801603])

In [11]:
NB.theta_

array([[ 1.00000000e+00,  4.27902240e-02,  3.38940937e-02,
        -9.80651731e-03, -1.05987780e-02,  4.36659878e-03,
         1.37184259e+00],
       [ 1.00000000e+00, -3.95463511e-02, -3.13254438e-02,
         5.83431953e-03,  3.11045365e-03, -6.50887574e-04,
         1.36320990e+00]])

In [12]:
NB.var_

array([[1.54306309e-09, 1.50355429e+00, 1.53246749e+00, 1.52305652e+00,
        1.60064884e+00, 1.44668659e+00, 7.57137793e-02],
       [1.54306309e-09, 1.51401364e+00, 1.48732877e+00, 1.51198994e+00,
        1.43804198e+00, 1.63638575e+00, 6.74751652e-02]])

In [15]:
X_train[L_train == 'Down']. mean ()

intercept    1.000000
Lag1         0.042790
Lag2         0.033894
Lag3        -0.009807
Lag4        -0.010599
Lag5         0.004367
Volume       1.371843
dtype: float64

In [16]:
X_train[L_train == 'Down'].var(ddof =0)

intercept    0.000000
Lag1         1.503554
Lag2         1.532467
Lag3         1.523057
Lag4         1.600649
Lag5         1.446687
Volume       0.075714
dtype: float64

In [17]:
nb_labels = NB.predict(X_test)
confusion_table(nb_labels , L_test)

Truth,Down,Up
Predicted,,
Down,85,111
Up,26,30


In [18]:
NB.predict_proba(X_test)[:5]

array([[0.52512367, 0.47487633],
       [0.46691112, 0.53308888],
       [0.48813902, 0.51186098],
       [0.50349077, 0.49650923],
       [0.50231369, 0.49768631]])

---